In [ ]:
import numpy as np
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

# Generate some example data
np.random.seed(42)
num_days = 100
num_stocks = 5
stock_prices = pd.read_csv('tr_eikon_eod_data.csv', parse_dates=True)
#stock_prices = stock_prices[['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GS.N']].dropna().to_numpy()
stock_prices = stock_prices['AAPL.O'].dropna().to_numpy()
stock_prices = stock_prices.reshape(-1, 1)
stock_prices = pd.DataFrame(stock_prices)



data = pd.read_csv('tr_eikon_eod_data.csv', parse_dates=True)
data = data[['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GS.N']]

# data = np.log(data / data.shift(1))
# data.dropna(inplace=True)

#  Split the data into training and testing sets
#train_data, test_data = train_test_split(stock_prices, test_size=0.2, random_state=42)
train_data, test_data = stock_prices[:1069], stock_prices[1069:]


test_data = data['GS.N'].dropna().to_numpy()
test_data = test_data.reshape(-1, 1)
test_data = pd.DataFrame(test_data)

# Define and train the HMM model
model = hmm.GaussianHMM(n_components=2, covariance_type="full", n_iter=100)
model.fit(train_data)

# Predict states for test data
predicted_states = model.predict(test_data)

# Initialize variables
current_balance = 0
position = None
transaction_cost = 0.01  # Assuming a transaction cost of 1%
value = 0

# Function to calculate profit based on transaction type
def calculate_profit(transaction_type, price, change = False):
    global current_balance, position, value
    price = float(price)
    if transaction_type == "buy":
        if change :
            position = "long" 
            current_balance -= price*value
        current_balance -= 100*price  # Buying for $100
    elif transaction_type == "sell":
        if change : 
            position = "short" 
            current_balance += price*value
        current_balance += 100*price  # Selling short for $100
    
    if change : 
        value = 100
    else :
        value += 100


# # Simulate transactions and calculate profit
# print(predicted_states.reshape(-1,1).ndim)
# print(test_data.to_numpy().ndim)
# print(np.hstack([predicted_states.reshape(-1,1), test_data.to_numpy()]))
        
for state, price in np.hstack([predicted_states.reshape(-1,1), test_data.to_numpy()]):
    print(state, price)
    try :
        price = float(price)
    except :
        continue
    print(price)
    if state == 0:  # Buy
        if position == "long":  # No position, buy
            calculate_profit("buy", price)
        if position == "short" or position is None:  # No position, buy
            calculate_profit("buy", price, True)
    else:  # Sell (short)
        if position == "short":  # Shorting when there's a position
            calculate_profit("sell", price)
        if position == "long" or position is None:  # Shorting when there's a position
            calculate_profit("sell", price, True)
    print(current_balance)

# Calculate final profit
multiplier = 0
if position == "long" :
    multiplier = 1
else :
    multiplier = -1
    
print(value, position)
final_balance = current_balance + test_data.iloc[-1, 0] * value * multiplier

print(f"Final balance: ${final_balance:.2f}")